In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

In [ ]:
# đọc dữ liệu từ file csv
df = pd.read_csv('/content/1.csv')

# Xóa hai dòng "KL" và "Thay đổi %" từ DataFrame dataSet
df = df.drop(columns=["KL", "% Thay đổi"])

# Hiển thị lại DataFrame sau khi xóa
print(df)

In [ ]:
df["Ngày"] = pd.to_datetime(df.Ngày,format="%d/%m/%Y")

In [ ]:
df.head()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Thấp'].value_counts()
    for x_label, grp in _df_12.groupby('Cao')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Cao')
_ = plt.ylabel('Thấp')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Cao'].value_counts()
    for x_label, grp in _df_11.groupby('Mở')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Mở')
_ = plt.ylabel('Cao')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Mở'].value_counts()
    for x_label, grp in _df_10.groupby('Lần cuối')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Lần cuối')
_ = plt.ylabel('Mở')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
from matplotlib.dates import YearLocator, DateFormatter, MonthLocator # Thêm MonthLocator vào để sửa lỗi

# Chuyển đổi cột "Ngày" sang dạng datetime
df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')

# Sắp xếp lại dữ liệu theo thứ tự thời gian
df = df.sort_values(by='Ngày')

# Chuyển đổi định dạng các cột giá thành số thực
df['Lần cuối'] = df['Lần cuối'].str.replace(',', '').astype(float)
df['Mở'] = df['Mở'].str.replace(',', '').astype(float)
df['Cao'] = df['Cao'].str.replace(',', '').astype(float)
df['Thấp'] = df['Thấp'].str.replace(',', '').astype(float)

# Lấy thông tin năm từ cột "Ngày"
df['Năm'] = df['Ngày'].dt.year

# Tạo đồ thị giá đóng cửa qua các năm
plt.figure(figsize=(10, 5))
plt.plot(df['Ngày'], df['Lần cuối'], label='Giá lần cuối', color='red')
plt.xlabel('Năm')
plt.ylabel('Giá lần cuối')
plt.title('Biểu đồ giá đóng cửa của VNM qua các năm')
plt.legend(loc='best')

# Định dạng đồ thị hiển thị các ngày tháng theo năm-tháng
years = YearLocator()
yearsFmt = DateFormatter('%Y')
months = MonthLocator()  # Thêm dòng này để khai báo MonthLocator
plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(yearsFmt)
plt.gca().xaxis.set_minor_locator(months)

plt.tight_layout()
plt.show()


In [ ]:
df1 = pd.DataFrame(df,columns=['Ngày','Lần cuối'])
df1.index = df1.Ngày
df1.drop('Ngày',axis=1,inplace=True)
df1

In [ ]:
# @title Lần cuối

from matplotlib import pyplot as plt
df1['Lần cuối'].plot(kind='hist', bins=20, title='Lần cuối')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Lần cuối

from matplotlib import pyplot as plt
df1['Lần cuối'].plot(kind='line', figsize=(8, 4), title='Lần cuối')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:

data = df1.values
train_data = data[:1500]
test_data = data[1500:]

In [ ]:
data

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
sc_train = sc.fit_transform(data)

In [ ]:
x_train,y_train=[],[]
for i in range(50,len(train_data)):
  x_train.append(sc_train[i-50:i,0])
  y_train.append(sc_train[i,0])

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = np.reshape(y_train,(y_train.shape[0],1))

In [ ]:
model = Sequential()

model.add(LSTM(units=128,input_shape=(x_train.shape[1],1),return_sequences=True))
model.add(LSTM(units=64))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
save_model = "/content/drive/MyDrive/dataset/save_model.hdf5"
best_model = ModelCheckpoint(save_model,monitor='loss',verbose=2,save_best_only=True,mode='auto')
model.fit(x_train,y_train,epochs=100,batch_size=50,verbose=2,callbacks=[best_model])

In [ ]:
y_train = sc.inverse_transform(y_train)
final_model = load_model("/content/drive/MyDrive/dataset/save_model.hdf5")
y_train_predict = final_model.predict(x_train)
y_train_predict = sc.inverse_transform(y_train_predict)

In [ ]:
test = df1[len(train_data)-50:].values
test = test.reshape(-1,1)
sc_test = sc.transform(test)

x_test = []
for i in range(50,test.shape[0]):
  x_test.append(sc_test[i-50:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))


y_test = data[1500:]
y_test_predict = final_model.predict(x_test)
y_test_predict = sc.inverse_transform(y_test_predict)

In [ ]:
train_data1 = df1[50:1500]
test_data1 = df1[1500:]

plt.figure(figsize=(24,8))
plt.plot(df1,label='Giá thực tế',color='red')
train_data1['Dự đoán'] = y_train_predict
plt.plot(train_data1['Dự đoán'],label='Giá dự đoán train',color='green')
test_data1['Dự đoán'] = y_test_predict
plt.plot(test_data1['Dự đoán'],label='Giá dự đoán test',color='blue')
plt.title('So sánh giá dự báo và giá thực tế')
plt.xlabel('Thời gian')
plt.ylabel('Giá lần cuối (VNĐ)')
plt.legend()
plt.show()

In [ ]:
print('Độ phù hợp tập train:',r2_score(y_train,y_train_predict))

print('Sai số tuyệt đối trung bình trên tập train (VNĐ):',mean_absolute_error(y_train,y_train_predict))

print('Phần trăm sai số tuyệt đối trung bình tập train:',mean_absolute_percentage_error(y_train,y_train_predict))

In [ ]:
train_data1

In [ ]:
from matplotlib import pyplot as plt
train_data1['Dự đoán'].plot(kind='line', figsize=(8, 4), title='Dự đoán')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Lần cuối

from matplotlib import pyplot as plt
train_data1['Lần cuối'].plot(kind='line', figsize=(8, 4), title='Lần cuối')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Dự đoán

from matplotlib import pyplot as plt
train_data1['Dự đoán'].plot(kind='hist', bins=20, title='Dự đoán')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Lần cuối

from matplotlib import pyplot as plt
train_data1['Lần cuối'].plot(kind='hist', bins=20, title='Lần cuối')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
print('Độ phù hợp tập test:',r2_score(y_test,y_test_predict))

print('Sai số tuyệt đối trung bình trên tập test (VNĐ):',mean_absolute_error(y_test,y_test_predict))

print('Phần trăm sai số tuyệt đối trung bình tập test:',mean_absolute_percentage_error(y_test,y_test_predict))

In [ ]:
test_data1

In [ ]:
next_date = df['Ngày'].iloc[-1] + pd.Timedelta(days=1)

next_date = pd.to_datetime(next_date)

next_closing_price = np.array([df['Lần cuối'].iloc[-1]])

next_closing_price_normalized = sc.transform(next_closing_price.reshape(-1, 1))


x_next = np.array([sc_train[-50:, 0]])
x_next = np.reshape(x_next, (x_next.shape[0], x_next.shape[1], 1))
y_next_predict = final_model.predict(x_next)
y_next_predict = sc.inverse_transform(y_next_predict)

df_next = pd.DataFrame({'Ngày': [next_date], 'Lần cuối': [y_next_predict[0][0]]})
df1 = pd.concat([df1, df_next])

plt.figure(figsize=(15, 5))
plt.plot(df1['Ngày'], df1['Lần cuối'], label='Giá thực tế', color='red')
plt.plot(train_data1.index, train_data1['Dự đoán'], label='Giá dự đoán train', color='green')
plt.plot(test_data1.index, test_data1['Dự đoán'], label='Giá dự đoán test', color='blue')
plt.scatter([next_date], [y_next_predict[0][0]], color='orange', label='Dự đoán ngày kế tiếp')
plt.xlabel('Thời gian')
plt.ylabel('Giá lần cuối (VNĐ)')
plt.title('So sánh giá dự báo và giá thực tế')
plt.legend()
plt.show()


In [ ]:
actual_closing_price = df['Lần cuối'].iloc[-1]

comparison_df = pd.DataFrame({'Ngày': [next_date], 'Giá dự đoán': [y_next_predict[0][0]], 'Giá ngày trước': [actual_closing_price]})

print(comparison_df)